In [232]:
import pandas as pd 
import pandas_profiling as pp
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings                        # To ignore any warnings warnings.filterwarnings("ignore")

In [243]:
cases=pd.read_csv("C:/Users/abhi/Downloads/covid_dataset/Facilities_in_Mumbai_COVID_19_Cases.csv")
wards=pd.read_csv("C:/Users/abhi/Downloads/covid_dataset/input_file_v1_dashboard.csv")
#state_geo_df=geopandas.read_file("C:/Users/abhi/Downloads/covid_dataset/BMC_Wards.geojson")

In [234]:
#cases=cases[pd.notnull(cases["Ward"])]
#cases.head()

In [249]:
pp.ProfileReport(cases)

Number of variables,5
Number of observations,25
Total Missing (%),25.6%
Total size in memory,1.1 KiB
Average record size in memory,43.2 B
Numeric,3
Categorical,1
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [250]:
pp.ProfileReport(wards)

Number of variables,15
Number of observations,25
Total Missing (%),0.0%
Total size in memory,3.0 KiB
Average record size in memory,123.2 B
Numeric,2
Categorical,0
Boolean,0
Date,0
Text (Unique),2
Rejected,11


In [244]:
df=pd.merge(left=cases, right=wards, left_on='Ward', right_on='Ward')
null_fill={'Number of Cases- Standalone Structure': 0, 'Number of Cases- Medium Congested': 0, 'Number of Cases- Very Congested Area': 0}
df=df.fillna(value=null_fill)
df["heat_value"]=round(((df["Number of Cases- Standalone Structure"]*1.5) + (df["Number of Cases- Medium Congested"]*2.6) + (df["Number of Cases- Very Congested Area"]*3.6))*100/df["Population"],2)
df["scaled_heat_value"]=(df["heat_value"]*1000).astype(int)
df.head(50)

,Ward,Number of Cases- Very Congested Area,Number of Cases- Medium Congested,Number of Cases- Standalone Structure,Date Entered,Country,State,City,Ward location,Population,...,Senior citizen %,Area,Educational institutes,Hospitals,Supermarts/malls,Critical congestion points,Commercial complexes,Residential areas,heat_value,scaled_heat_value
0,A,1.0,0.0,5.0,NaN,India,Maharashtra,Mumbai,Colaba,210847,...,0.14,12.50,54,40,20,0,0,0,0.01,10
1,B,3.0,0.0,12.0,NaN,India,Maharashtra,Mumbai,Sanhurst Road,140633,...,0.14,2.47,48,32,32,0,0,0,0.02,20
2,C,3.0,0.0,9.0,NaN,India,Maharashtra,Mumbai,Marine Lines,202922,...,0.14,1.78,43,23,21,0,0,0,0.01,10
3,D,3.0,3.0,18.0,NaN,India,Maharashtra,Mumbai,Grant Road,382841,...,0.14,6.63,23,43,21,0,0,0,0.01,10
4,E,2.0,3.0,5.0,NaN,India,Maharashtra,Mumbai,Byculla,440335,...,0.14,7.40,32,57,12,0,0,0,0.01,10
5,F/N,5.0,0.0,10.0,NaN,India,Maharashtra,Mumbai,Matunga,524393,...,0.14,12.98,53,32,23,0,0,0,0.01,10
6,F/S,1.0,2.0,11.0,NaN,India,Maharashtra,Mumbai,Parel,396122,...,0.14,14.00,42,43,21,0,0,0,0.01,10
7,G/S,24.0,3.0,28.0,NaN,India,Maharashtra,Mumbai,Elphinstone,457931,...,0.14,10.00,54,21,21,0,0,0,0.03,30
8,G/N,6.0,0.0,12.0,NaN,India,Maharashtra,Mumbai,Dadar/Plaza,582007,...,0.14,9.07,33,33,23,0,0,0,0.01,10
9,H/E,11.0,1.0,16.0,NaN,India,Maharashtra,Mumbai,Khar/Santacruz,580835,...,0.14,13.53,54,23,32,0,0,0,0.01,10


In [245]:
import json
with open('C:/Users/abhi/Downloads/covid_dataset/BMC_Wards.geojson') as f:
    mumbai_geojson = json.load(f)

In [251]:
import plotly.express as px
px.set_mapbox_access_token("pk.eyJ1IjoiYWJoaTkyNTQ0IiwiYSI6ImNrNmN4eGUwbjAwdDgza3Fvb3Z3MWFnbnMifQ.E9jcJwsGsiw6JJxHGzCb3Q")

mcgm=['rgb(0,255,0)',  'rgb(0,0,255)', 'rgb(255,100,0)',  'rgb(255,0,0)']
#myscalelight=['rgb(144,238,144)',  'rgb(0,191,255)', 'rgb(255,165,0)',  'rgb(255,69,0)']
#customscale=[(0.00, "green"),(0.01, "blue"),(0.02, "orange"), (0.02, "red")]

fig = px.choropleth_mapbox(df, geojson=mumbai_geojson, color="heat_value",
                           color_continuous_scale=mcgm,
                           #color_continuous_scale="rdylgn_r",
                           locations="Ward", featureidkey="properties.name",
                           center={"lat": 19.0760, "lon": 72.8777},
                           mapbox_style="carto-positron", zoom=9.75)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [246]:
geo_df=gpd.read_file("C:/Users/abhi/Downloads/covid_dataset/BMC_Wards.geojson")

In [247]:
geo_df["geometry"]=geo_df['geometry'].centroid
#geo_df.rename(columns={'geometry': 'borders'}).set_geometry('borders')
final_df=pd.merge(left=geo_df,right=df, left_on='name',right_on='Ward')

In [248]:
final_df[['gid','name','geometry','heat_value','scaled_heat_value']].to_file("C:/Users/abhi/Downloads/covid_dataset/BMC_Wards_output.geojson", driver='GeoJSON')